## This notebook demonstrates how to run the [GARPOS](https://github.com/s-watanabe-jhod/garpos) software in the EarthScope GeoLab cloud environment.  
06-05-2025

### GeoLab Basics

GeoLab is the name of EarthScope's Jupyter Hub, which allows users to run code via Jupyter notebooks (or in a terminal) on EarthScope resources in AWS.  

The `sfg-geolab` image comes pre-loaded with the software needed to process GNSS-A data. Each user has their own home directory `/home/jovyan` that is mounted to 
the image and any files kept there are accessibile only to that user and remain during future sessions.  

### Data 
For the purposes of this GARPOS demo, we have pre-processed some `cascadia-gorda` data and stored the shot data in an S3 bucket accessible to all users of the GeoLab hub.

Raw data is available via authenticated https from [https://data.earthscope.org/archive/seafloor](https://data.earthscope.org/archive/seafloor).  Where pre-processed data is not available, or you want to create your own, you would use the `SV3_data_preprocessing.ipynb` notebook first to generate a copy within your user directory on the hub.  

### Metadata
In order to simplify our interactions with the various metadata formats, we are translating all metadata into `site` and `vessel` json. We store and load these metadata from the S3 archive [https://data.earthscope.org/archive/seafloor/metadata](https://data.earthscope.org/archive/seafloor/metadata)

### Results
Results are stored per GARPOS run under `/home/joyvan/data/sfg/[network]/[station]/GARPOS/[campaign]/[survey]/results/`. 

### Logs
Some log output is piped to the notebook.  Additionally those logs plus debug logs are written to `/home/joyvan/data/sfg/[network]/[station]/logs`. 

In [ ]:
#Load required modules
from es_sfgtools.data_mgmt.data_handler import DataHandler
from es_sfgtools.utils.archive_pull import load_site_metadata
import os

# from sfg_metadata import GEOLAB_CATALOG
from pathlib import Path
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [32,18]

## Select the data you are interested in running through GARPOS
Currently we have pre-processed the following campaigns and made shot data available to this environment in a shared S3 bucket.

|  | GCC1 | NBR1 | NCC1 |
|---|---|---|---|
| **2022** |2022_A_1065 |   |   |
| **2023** |   | 2023_A_1063 | 2023_A_1063 |
| **2024** |   |   |  |


In [ ]:
# Enter the network, station, and campaign of interest
network = "cascadia-gorda"
station = "NBR1"
campaign_name = '2023_A_1063'

# This flag selects using our shared preprocessed data.  If false, you must first preprocess your own data 
# and have the shotdata available in your hub data directory
use_shared_data = True


#Initialize a data handler object for the selected campaign, this builds local directory structures and tracks 
#where data is written.  Do not edit this code.
main_dir = Path.home()/"data"/"sfg"
if use_shared_data:
    dh = DataHandler(directory=main_dir)
else:
    dh = DataHandler(directory=main_dir)

# Set the station and campaig
dh.change_working_station(network=network, station=station, campaign=campaign_name)

# Add site ctds to catalog
dh.add_ctds_to_catalog()

## Load the required processing metadata for the selected station:
This includes
- station metadata
  - array center
  - benchmark/transponder info
  - campaign/survey times
- vessel metadata
  - ATD offset

In [ ]:
#Load the metadata and view what surveys are in the selected campaign
station_meta = load_site_metadata(network, station)
for campaign in station_meta.campaigns:
    if campaign.name == campaign_name:
        surveys = campaign.surveys
        print(f"The campaign {station} {campaign.name} contains {len(surveys)} survey patterns")
        for survey in surveys:
            print(f"  Survey Name: {survey.id}")
            print(f"     Survey Type: {survey.type} {survey.notes if survey.notes is not None else ''}")
            print(f"     {survey.start} to {survey.end}")

## Set up the GARPOS runs

This section prepares the input files needed to run GARPOS.  We are batching GARPOS runs by survey, so this step generates a set of files for each of the surveys listed above.

Data files (input files and results) will be stored in the locally mounted filesystem under 

`/home/joyvan/data/sfg/[network]/[station]/GARPOS/[campaign_name]/[survey_name]`

In [ ]:
# Initialize a garpos handler object, this is used to prep data and run the GARPOS software
gp_handler = dh.get_garpos_handler(site_data=station_meta)

In [ ]:
# Prepare a sound speed profile input file.  Looks for a CTD in our s3 archive. Can also override with a local CTD file.
gp_handler.load_sound_speed_data()

In [ ]:
# If you wish to override the default data filters, you can do so here. 
# Once added, pass the custom_filters dictionary to the prep_shotdata function below
custom_filters = {'acoustic_filters': {'enabled': False, 'level': 'OK'}, 
                  'ping_replies': {'enabled': False, 'min_replies': 3}, 
                  'max_distance_from_center': {'enabled': False, 'max_distance_m': 150}, 
                  'pride_residuals': {'enabled': False, 'max_residual_mm': 5}}

# Load the shot_data from the shared S3 bucket and write out the garpos obs input files
gp_handler.prep_shotdata(overwrite=True)
# customs_filters=custom_filters


In [ ]:
#View the default GARPOS inversion parameters
gp_handler.garpos_fixed.inversion_params.show_params()

In [ ]:
# Modify GARPOS parameters if desired
update_dict = {
                "rejectcriteria": 2.5,
                "log_lambda":[0],
                "maxloop": 5 #to speed things up for the sake of demo
            }
gp_handler.set_inversion_params(update_dict)
gp_handler.garpos_fixed.inversion_params.show_params()

## Run GARPOS
This will loop through all the surveys in the campaign and run GARPOS on each.  

You can change the above inversion parameters and run again to compare results, but make sure to increment run_id so that the results are stored in a new directory.

In [ ]:
run_id = 1
survey_number = 1
survey_id=f"{campaign_name}_{survey_number}"

In [ ]:
gp_handler.run_garpos(survey_id=survey_id,
                      run_id=run_id,
                      override=True)

## Plot results

In [ ]:
#%matplotlib widget
residual_filter = 25
gp_handler.plot_ts_results(survey_id=survey_id,
                           run_id=run_id,
                           res_filter=residual_filter
                          )